In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os

import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras.applications.resnet50 import ResNet50, preprocess_input as resnet50_preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess_input
from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.utils.np_utils import to_categorical

from constants import img_size, source_dir_dev, source_dir_train

Using TensorFlow backend.


In [2]:
def generator(datagen, data_dir, batch_size):
    return datagen.flow_from_directory(
        data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=0,
    )

In [3]:
def base_model(model_constructor, name, preprocessor, pooling, verbose):
    datagen = ImageDataGenerator(
        preprocessing_function=preprocessor,
    )
    
    model = model_constructor(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3), pooling=pooling)
    
    if verbose == True:
        model.summary()
        
    return model

In [4]:
def paths(name):
    paths_to_create = [
        './data/features/' + name,
    ]
    
    for path in paths_to_create:
        if not os.path.exists(path):
            os.makedirs(path)
            
    return paths_to_create[0]

In [5]:
def extractor(model_constructor, name, preprocessor, pooling, batch_size, verbose = True):
    model = base_model(model_constructor, name, preprocessor, pooling, verbose)

    datagen = ImageDataGenerator(preprocessing_function=preprocessor)
    generator_train = generator(datagen, source_dir_train, batch_size)
    generator_dev = generator(datagen, source_dir_dev, batch_size)
    
    path = paths(name)
    np.save(path + '/filenames_train.npy', generator_train.filenames)
    np.save(path + '/classes_train.npy', generator_train.classes)
    np.save(path + '/filenames_dev.npy', generator_dev.filenames)
    np.save(path + '/classes_dev.npy', generator_dev.classes)
    
    #train features
    features_train = model.predict_generator(generator_train, len(generator_train), verbose=1)
    np.save(path + '/features_train.npy', features_train)
    
    #dev features
    features_dev = model.predict_generator(generator_dev, len(generator_dev), verbose=1)
    np.save(path + '/features_dev.npy', features_dev)

In [6]:
extractor(ResNet50, 'res_net', resnet50_preprocess_input, 'avg', 32)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Found 7227 images belonging to 196 classes.
Found 917 images belonging to 196 classes.
13/29 [============>.................] - ETA: 4:23

/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 16384000 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 808583168 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2949120 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 285413634 bytes but only got 0. Skipping tag 780
  " Skipping tag %s" % (size, len(data), tag))
/opt/conda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expectin

29/29 [==============================] - 467s 16s/step
